### Calibrated 3D Face Reconstruction from Stereo Images

In [1]:
import open3d as o3d
import numpy as np
import cv2 as cv
import json
import os

# Coordinate transformation from Blender to OpenCV
def blender_to_opencv_transform():
    return np.array([
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1]
    ])

# Load grayscale images from three cameras
def load_grayscale_image(path):
    img = cv.imread(path, cv.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Image not found: {path}")
    return img

img_left = load_grayscale_image("5/Camera_Left.png")
img_front = load_grayscale_image("5/Camera_Front.png")
img_right = load_grayscale_image("5/Camera_Right.png")

# Extract SIFT features
sift = cv.SIFT_create(nfeatures=20000, contrastThreshold=0.005, edgeThreshold=10, sigma=1.2)
kpL, desL = sift.detectAndCompute(img_left, None)
kpF, desF = sift.detectAndCompute(img_front, None)
kpR, desR = sift.detectAndCompute(img_right, None)

# Load camera parameters
with open("5/camera_params.json", "r") as f:
    params = json.load(f)

# Access camera parameter as dictionary
def get_params(cam_name):
    return params[cam_name]

# Use extrinsic matrix directly
def parse_blender_camera(cam):
    extrinsic = np.array(cam["extrinsic"])  # 4x4 Blender world-to-camera matrix
    R_blender = extrinsic[:3, :3]
    t_blender = extrinsic[:3, 3].reshape(3, 1)

    R_transform = blender_to_opencv_transform()

    R_world_to_cam = R_blender
    R = R_transform @ R_world_to_cam

    C_world = -R_world_to_cam.T @ t_blender
    C = R_transform @ C_world

    return R, C

# Parse camera extrinsics
camL = get_params("Camera_Left")
camF = get_params("Camera_Front")
camR = get_params("Camera_Right")

R_world_to_L, C_L = parse_blender_camera(camL)
R_world_to_F, C_F = parse_blender_camera(camF)
R_world_to_R, C_R = parse_blender_camera(camR)

# Compute relative pose
def compute_relative_pose(R1, C1, R2, C2):
    R_rel = R2 @ R1.T
    t_rel = R2 @ (C1 - C2)
    return R_rel, t_rel

R_F_to_L, t_F_to_L = compute_relative_pose(R_world_to_F, C_F, R_world_to_L, C_L)
R_F_to_R, t_F_to_R = compute_relative_pose(R_world_to_F, C_F, R_world_to_R, C_R)

# Intrinsics
K = np.array([
    [2666.6667, 0, 960.0],
    [0, 2250.0, 540.0],
    [0, 0, 1]
])

# Projection matrix builder
def build_projection(R_rel, t_rel):
    return K @ np.hstack([R_rel, t_rel])

# Projection matrices
P_F = K @ np.hstack([np.eye(3), np.zeros((3, 1))])
P_L = build_projection(R_F_to_L, t_F_to_L)
P_R = build_projection(R_F_to_R, t_F_to_R)

# Feature matcher and RANSAC filter
def get_inliers(des1, des2, kp1, kp2):
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    good = [m for m, n in matches if m.distance < 0.7 * n.distance]
    if not good:
        return np.array([]), np.array([])
    pts1 = np.float32([kp1[m.queryIdx].pt for m in good])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good])
    F, mask = cv.findFundamentalMat(pts1, pts2, cv.FM_RANSAC, 1.0, 0.99)
    if F is None or mask.sum() < 4:
        return np.array([]), np.array([])
    return pts1[mask.ravel() == 1], pts2[mask.ravel() == 1]

# Match and triangulate
print("Matching pairs...")
ptsF_L, ptsL_F = get_inliers(desF, desL, kpF, kpL)
print(f"Left-Front correspondences: {len(ptsL_F)}")
ptsF_R, ptsR_F = get_inliers(desF, desR, kpF, kpR)
print(f"Front-Right correspondences: {len(ptsF_R)}")
ptsL_R, ptsR_L = get_inliers(desL, desR, kpL, kpR)
print(f"Left-Right correspondences: {len(ptsL_R)}")

# Triangulate points
def triangulate_points(P1, P2, pts1, pts2):
    if len(pts1) < 4 or len(pts2) < 4:
        return np.array([])
    pts4d = cv.triangulatePoints(P1, P2, pts1.T, pts2.T)
    pts3d = (pts4d[:3] / pts4d[3]).T
    valid = (pts3d[:, 2] > 0.1) & (pts3d[:, 2] < 10.0)
    return pts3d[valid]

print("Triangulating...")
points_FL = triangulate_points(P_F, P_L, ptsF_L, ptsL_F)
points_FR = triangulate_points(P_F, P_R, ptsF_R, ptsR_F)
points_LR = triangulate_points(P_L, P_R, ptsL_R, ptsR_L)

# Combine points
all_points = np.vstack([points_FL, points_FR, points_LR])

# Transform to Blender coordinates
transform = blender_to_opencv_transform().T
points_blender = all_points @ transform

# Create and save colored point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_blender)
red_color = np.tile([1.0, 0.0, 0.0], (len(points_blender), 1))
pcd.colors = o3d.utility.Vector3dVector(red_color)

o3d.io.write_point_cloud("reconstruction.ply", pcd)
print(f" Reconstructed {len(points_blender)} points and saved to reconstruction.ply")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Matching pairs...
Left-Front correspondences: 6111
Front-Right correspondences: 5862
Left-Right correspondences: 4168
Triangulating...
 Reconstructed 16141 points and saved to reconstruction.ply


### Adding Texiture from Front image

In [2]:
def create_textured_mesh(points, camera_params, texture_img_path):
    # Convert 3D points into a point cloud and estimate normals
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.estimate_normals()
    pcd.orient_normals_consistent_tangent_plane(k=15)

    # Generate a mesh using Poisson surface reconstruction
    mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)
    mesh.remove_duplicated_vertices()
    mesh.remove_degenerate_triangles()
    mesh.remove_unreferenced_vertices()
    mesh = mesh.filter_smooth_simple(number_of_iterations=1)

    # Retrieve camera intrinsic and extrinsic parameters
    K = camera_params['K']
    R = camera_params['R']
    T = camera_params['T'].reshape(3, 1)
    width = camera_params['width']
    height = camera_params['height']

    # Project 3D mesh vertices into 2D image space
    vertices = np.asarray(mesh.vertices)
    cam_coords = (R @ vertices.T + T).T
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    x_proj = (fx * cam_coords[:, 0] / cam_coords[:, 2]) + cx
    y_proj = (fy * cam_coords[:, 1] / cam_coords[:, 2]) + cy

    # Compute UV texture coordinates (with U flipped for image alignment)
    u = np.clip(1 - (x_proj / width), 0, 1)
    v = np.clip(y_proj / height, 0, 1)

    # Assign UV coordinates per triangle vertex
    triangle_uvs = []
    for tri in np.asarray(mesh.triangles):
        triangle_uvs.extend([
            [u[tri[0]], v[tri[0]]],
            [u[tri[1]], v[tri[1]]],
            [u[tri[2]], v[tri[2]]]
        ])
    mesh.triangle_uvs = o3d.utility.Vector2dVector(np.array(triangle_uvs))
    mesh.triangle_material_ids = o3d.utility.IntVector([0] * len(mesh.triangles))

    # Load and process the texture image
    texture_img = o3d.io.read_image(texture_img_path)
    texture_np = np.asarray(texture_img)
    if texture_np.ndim == 3 and texture_np.shape[2] == 4:
        texture_np = texture_np[:, :, :3]  # Remove alpha channel
    elif texture_np.ndim == 2:
        texture_np = np.stack([texture_np] * 3, axis=-1)  # Grayscale to RGB
    texture_np = np.ascontiguousarray(texture_np)  # Ensure memory layout
    mesh.textures = [o3d.geometry.Image(texture_np)]

    return mesh

# Define intrinsic and extrinsic parameters for the front camera
front_cam_params = {
    'K': K,
    'R': np.eye(3),  # No rotation (identity)
    'T': np.zeros((3, 1)),  # No translation
    'width': 1920,
    'height': 1080
}

# Generate the textured mesh using front camera projection and texture
textured_mesh = create_textured_mesh(
    points_blender,
    front_cam_params,
    "5/Camera_Front.png"
)

# Post-process mesh normals for better visualization and export
textured_mesh.compute_vertex_normals()
textured_mesh.compute_triangle_normals()

# Save the textured mesh  OBJ formats with UVs
o3d.io.write_triangle_mesh("reconstruction_textured.obj", textured_mesh, write_triangle_uvs=True)

print(f"Saved textured mesh with {len(all_points)} points")

[Open3D WARNING] Write OBJ can not include triangle normals.
Saved textured mesh with 16141 points


### Reprojection Error Analysis

In [3]:
import open3d as o3d
import numpy as np
import cv2 as cv
import json

# Coordinate transformation from Blender to OpenCV
def blender_to_opencv_transform():
    return np.array([
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1]
    ])

# Load grayscale image
def load_grayscale_image(path):
    img = cv.imread(path, cv.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f" Image not found: {path}")
    return img

# Load images
img_left = load_grayscale_image("5/Camera_Left.png")
img_front = load_grayscale_image("5/Camera_Front.png")
img_right = load_grayscale_image("5/Camera_Right.png")

# SIFT features
sift = cv.SIFT_create(nfeatures=20000, contrastThreshold=0.005, edgeThreshold=10, sigma=1.2)
kpL, desL = sift.detectAndCompute(img_left, None)
kpF, desF = sift.detectAndCompute(img_front, None)
kpR, desR = sift.detectAndCompute(img_right, None)

# Load camera parameters
with open("5/camera_params.json", "r") as f:
    params = json.load(f)

def get_params(cam_name):
    return params[cam_name]

# Convert Blender camera to OpenCV format
def parse_blender_camera(cam):
    extrinsic = np.array(cam["extrinsic"])
    R_blender = extrinsic[:3, :3]
    t_blender = extrinsic[:3, 3].reshape(3, 1)
    R_transform = blender_to_opencv_transform()
    R = R_transform @ R_blender
    C_world = -R_blender.T @ t_blender
    C = R_transform @ C_world
    return R, C

# Camera poses
camL = get_params("Camera_Left")
camF = get_params("Camera_Front")
camR = get_params("Camera_Right")
R_world_to_L, C_L = parse_blender_camera(camL)
R_world_to_F, C_F = parse_blender_camera(camF)
R_world_to_R, C_R = parse_blender_camera(camR)

# Relative poses
def compute_relative_pose(R1, C1, R2, C2):
    R_rel = R2 @ R1.T
    t_rel = R2 @ (C1 - C2)
    return R_rel, t_rel

R_F_to_L, t_F_to_L = compute_relative_pose(R_world_to_F, C_F, R_world_to_L, C_L)
R_F_to_R, t_F_to_R = compute_relative_pose(R_world_to_F, C_F, R_world_to_R, C_R)
R_L_to_R, t_L_to_R = compute_relative_pose(R_world_to_L, C_L, R_world_to_R, C_R)

# Intrinsics
K = np.array([
    [2666.6667, 0, 960.0],
    [0, 2250.0, 540.0],
    [0, 0, 1]
])

# Build projection matrix
def build_projection(R_rel, t_rel):
    return K @ np.hstack([R_rel, t_rel])

# Projections
P_F = K @ np.hstack([np.eye(3), np.zeros((3, 1))])
P_L = build_projection(R_F_to_L, t_F_to_L)
P_R = build_projection(R_F_to_R, t_F_to_R)
P_L_ref = K @ np.hstack([np.eye(3), np.zeros((3, 1))])  # For Left-Right pair
P_R_from_L = build_projection(R_L_to_R, t_L_to_R)

# Feature matching and RANSAC
def get_inliers(des1, des2, kp1, kp2):
    bf = cv.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    good = [m for m, n in matches if m.distance < 0.7 * n.distance]
    if not good:
        return np.array([]), np.array([])
    pts1 = np.float32([kp1[m.queryIdx].pt for m in good])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in good])
    F, mask = cv.findFundamentalMat(pts1, pts2, cv.FM_RANSAC, 1.0, 0.99)
    if F is None or mask.sum() < 4:
        return np.array([]), np.array([])
    return pts1[mask.ravel() == 1], pts2[mask.ravel() == 1]

# Triangulation with filtering
def triangulate_points(P1, P2, pts1, pts2):
    if len(pts1) < 4 or len(pts2) < 4:
        return np.array([]), np.array([])
    pts4d = cv.triangulatePoints(P1, P2, pts1.T, pts2.T)
    pts3d = (pts4d[:3] / pts4d[3]).T
    valid = (pts3d[:, 2] > 0.1) & (pts3d[:, 2] < 10.0)
    return pts3d[valid], pts1[valid]

# Match and triangulate
print("Matching pairs...")
ptsF_L, ptsL_F = get_inliers(desF, desL, kpF, kpL)
print(f"Left-Front matches: {len(ptsF_L)}")
ptsF_R, ptsR_F = get_inliers(desF, desR, kpF, kpR)
print(f"Front-Right matches: {len(ptsF_R)}")
ptsL_R, ptsR_L = get_inliers(desL, desR, kpL, kpR)
print(f"Left-Right matches: {len(ptsL_R)}")

print("Triangulating...")
points_FL, ptsF_L_valid = triangulate_points(P_F, P_L, ptsF_L, ptsL_F)
points_FR, ptsF_R_valid = triangulate_points(P_F, P_R, ptsF_R, ptsR_F)
points_LR, ptsL_R_valid = triangulate_points(P_L_ref, P_R_from_L, ptsL_R, ptsR_L)

# Combine and transform points to Blender space
all_points = np.vstack([points_FL, points_FR, points_LR])
transform = blender_to_opencv_transform().T
points_blender = all_points @ transform

# Save point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_blender)
red_color = np.tile([1.0, 0.0, 0.0], (len(points_blender), 1))
pcd.colors = o3d.utility.Vector3dVector(red_color)
o3d.io.write_point_cloud("reconstruction.ply", pcd)
print(f"Reconstructed {len(points_blender)} points and saved to reconstruction.ply")

# Reprojection error evaluation
def compute_reprojection_error(pts_3d, pts_2d, R, C, K):
    if pts_3d.shape[0] == 0 or pts_2d.shape[0] == 0:
        return None, None
    t = -R @ C
    rvec, _ = cv.Rodrigues(R)
    projected, _ = cv.projectPoints(pts_3d, rvec, t, K, None)
    projected = projected.squeeze()
    error = np.linalg.norm(projected - pts_2d, axis=1)
    return error.mean(), error

def evaluate_reconstruction():
    print("\n Evaluating Reprojection Errors:")

    err_FL, _ = compute_reprojection_error(points_FL, ptsF_L_valid, np.eye(3), np.zeros((3, 1)), K)
    if err_FL is not None:
        print(f"Front-Left reprojection error: {err_FL:.2f} pixels")

    err_FR, _ = compute_reprojection_error(points_FR, ptsF_R_valid, np.eye(3), np.zeros((3, 1)), K)
    if err_FR is not None:
        print(f"Front-Right reprojection error: {err_FR:.2f} pixels")

    err_LR, _ = compute_reprojection_error(points_LR, ptsL_R_valid, np.eye(3), np.zeros((3, 1)), K)
    if err_LR is not None:
        print(f"Left-Right reprojection error: {err_LR:.2f} pixels")

evaluate_reconstruction()

Matching pairs...
Left-Front matches: 6111
Front-Right matches: 5862
Left-Right matches: 4168
Triangulating...
Reconstructed 16141 points and saved to reconstruction.ply

 Evaluating Reprojection Errors:
Front-Left reprojection error: 0.06 pixels
Front-Right reprojection error: 0.06 pixels
Left-Right reprojection error: 0.06 pixels


### Calibrated 3D Face Reconstruction and Evaluation from Stereo Images

In [1]:
import open3d as o3d
import numpy as np
import cv2 as cv
import json
import os
import pyexr
from scipy.spatial import KDTree

def blender_to_opencv_transform():
    return np.array([
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1]
    ])

def load_grayscale_image(path):
    img = cv.imread(path, cv.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Image not found: {path}")
    return img

def load_exr_depth(path):
    try:
        exr = pyexr.read(path)
        if exr.ndim == 3:
            return exr[:, :, 0]
        return exr
    except Exception as e:
        print(f"Failed to load {path}: {e}")
        return None

def backproject_depth_map(depth_map, K, cam_to_world):
    h, w = depth_map.shape
    u, v = np.meshgrid(np.arange(w), np.arange(h))
    valid_depth = (depth_map > 0) & (depth_map < 100) & np.isfinite(depth_map)

    if not np.any(valid_depth):
        return np.array([])

    u_valid = u[valid_depth]
    v_valid = v[valid_depth]
    depths = depth_map[valid_depth]

    x = (u_valid - K[0, 2]) * depths / K[0, 0]
    y = (v_valid - K[1, 2]) * depths / K[1, 1]
    z = depths

    points_cam = np.vstack([x, y, z]).T
    points_cam_hom = np.hstack([points_cam, np.ones((len(points_cam), 1))])
    points_world = (cam_to_world @ points_cam_hom.T).T[:, :3]
    return points_world

def evaluate_reconstruction(recon_points, camera_params):
    """Evaluate reconstructed points against ground truth depth maps"""
    all_gt_points = []
    
    for cam_name, params in camera_params.items():
        try:
            # Get camera parameters
            intrinsic = np.array(params['intrinsic'])
            extrinsic = np.array(params['extrinsic'])
            depth_path = os.path.join("0", params['depth_path'])
            
            # Load depth map
            depth_map = load_exr_depth(depth_path)
            if depth_map is None:
                continue
                
            # Convert to single channel 
            if depth_map.ndim == 3:
                depth_map = depth_map[:, :, 0]
                
            # Compute camera to world transform
            cam_to_world = np.linalg.inv(extrinsic)
            
            # Backproject depth to 3D points
            gt_points = backproject_depth_map(depth_map, intrinsic, cam_to_world)
            if len(gt_points) > 0:
                all_gt_points.append(gt_points)
                print(f"{cam_name}: Generated {len(gt_points)} ground truth points")
        except Exception as e:
            print(f"Error processing {cam_name}: {e}")

    if not all_gt_points:
        print("No valid ground truth points")
        return None
        
    all_gt_points = np.vstack(all_gt_points)
    print(f"Total ground truth points: {len(all_gt_points)}")

    # Downsample if too many points
    if len(all_gt_points) > 1000000:
        print("Downsampling ground truth points for efficiency")
        indices = np.random.choice(len(all_gt_points), 1000000, replace=False)
        all_gt_points = all_gt_points[indices]

    # Build KDTree for evaluation
    gt_tree = KDTree(all_gt_points)
    distances, _ = gt_tree.query(recon_points, k=1, workers=-1)
    
    # Filter valid matches
    valid_mask = distances < 0.1  # 1 cm threshold
    valid_distances = distances[valid_mask]
    
    if len(valid_distances) == 0:
        print("No valid matches found")
        return None
        
 # Compute metrics with finer inlier thresholds
    metrics = {
        'mean_error': np.mean(valid_distances),
        'median_error': np.median(valid_distances),
        'std_error': np.std(valid_distances),
        'rmse': np.sqrt(np.mean(np.square(valid_distances))),
        'inlier_ratio_1mm': np.mean(valid_distances < 0.001),
        'inlier_ratio_1cm': np.mean(valid_distances < 0.01),
        'inlier_ratio_5cm': np.mean(valid_distances < 0.05),
        'coverage': len(valid_distances) / len(recon_points),
        'points_matched': len(valid_distances),
        'total_points': len(recon_points)
    }

    # Print results in centimeters 
    print("\n" + "-" * 60)
    print("Evaluation Metrics (units: centimeters)")
    print("-" * 60)
    print(f"{'Matched Points':20s}: {metrics['points_matched']}/{metrics['total_points']}")
    print(f"{'Mean Error':20s}: {metrics['mean_error'] * 100:.3f} cm")
    print(f"{'Median Error':20s}: {metrics['median_error'] * 100:.3f} cm")
    print(f"{'RMSE':20s}: {metrics['rmse'] * 100:.3f} cm")
    print(f"{'Std Dev':20s}: {metrics['std_error'] * 100:.3f} cm")
    print(f"{'Inlier Ratio (<1mm)':20s}: {metrics['inlier_ratio_1mm'] * 100:.2f}%")
    print(f"{'Inlier Ratio (<1cm)':20s}: {metrics['inlier_ratio_1cm'] * 100:.2f}%")
    print(f"{'Inlier Ratio (<5cm)':20s}: {metrics['inlier_ratio_5cm'] * 100:.2f}%")
    print(f"{'Coverage':20s}: {metrics['coverage'] * 100:.2f}%")
    print("-" * 60)

    return metrics

# Main reconstruction pipeline
if __name__ == "__main__":
    # Load images
    img_left = load_grayscale_image("5/Camera_Left.png")
    img_front = load_grayscale_image("5/Camera_Front.png")
    img_right = load_grayscale_image("5/Camera_Right.png")

    # Detect features
    sift = cv.SIFT_create(nfeatures=20000, contrastThreshold=0.005, edgeThreshold=10, sigma=1.2)
    kpL, desL = sift.detectAndCompute(img_left, None)
    kpF, desF = sift.detectAndCompute(img_front, None)
    kpR, desR = sift.detectAndCompute(img_right, None)

    # Load camera parameters
    with open("5/camera_params.json", "r") as f:
        params = json.load(f)

    def get_params(cam_name):
        return params[cam_name]

    def parse_blender_camera(cam):
        extrinsic = np.array(cam["extrinsic"])
        R_blender = extrinsic[:3, :3]
        t_blender = extrinsic[:3, 3].reshape(3, 1)

        R_transform = blender_to_opencv_transform()
        R = R_transform @ R_blender
        C_world = -R_blender.T @ t_blender
        C = R_transform @ C_world

        return R, C, extrinsic  # extrinsic is Blender world-to-camera

    camL = get_params("Camera_Left")
    camF = get_params("Camera_Front")
    camR = get_params("Camera_Right")

    R_world_to_L, C_L, extrinsic_L = parse_blender_camera(camL)
    R_world_to_F, C_F, extrinsic_F = parse_blender_camera(camF)
    R_world_to_R, C_R, extrinsic_R = parse_blender_camera(camR)

    def compute_relative_pose(R1, C1, R2, C2):
        R_rel = R2 @ R1.T
        t_rel = R2 @ (C1 - C2)
        return R_rel, t_rel

    R_F_to_L, t_F_to_L = compute_relative_pose(R_world_to_F, C_F, R_world_to_L, C_L)
    R_F_to_R, t_F_to_R = compute_relative_pose(R_world_to_F, C_F, R_world_to_R, C_R)

    # Use ACTUAL intrinsics from camera parameters
    K_front = np.array(camF["intrinsic"])
    K_left = np.array(camL["intrinsic"])
    K_right = np.array(camR["intrinsic"])

    def build_projection(R_rel, t_rel, K):
        return K @ np.hstack([R_rel, t_rel])

    P_F = K_front @ np.hstack([np.eye(3), np.zeros((3, 1))])
    P_L = build_projection(R_F_to_L, t_F_to_L, K_left)
    P_R = build_projection(R_F_to_R, t_F_to_R, K_right)

    def get_inliers(des1, des2, kp1, kp2):
        bf = cv.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)
        good = [m for m, n in matches if m.distance < 0.7 * n.distance]
        if not good:
            return np.array([]), np.array([])
        pts1 = np.float32([kp1[m.queryIdx].pt for m in good])
        pts2 = np.float32([kp2[m.trainIdx].pt for m in good])
        F, mask = cv.findFundamentalMat(pts1, pts2, cv.FM_RANSAC, 1.0, 0.99)
        if F is None or mask.sum() < 4:
            return np.array([]), np.array([])
        return pts1[mask.ravel() == 1], pts2[mask.ravel() == 1]

    print("Matching pairs...")
    ptsF_L, ptsL_F = get_inliers(desF, desL, kpF, kpL)
    print(f"Left-Front correspondences: {len(ptsL_F)}")
    ptsF_R, ptsR_F = get_inliers(desF, desR, kpF, kpR)
    print(f"Front-Right correspondences: {len(ptsF_R)}")
    ptsL_R, ptsR_L = get_inliers(desL, desR, kpL, kpR)
    print(f"Left-Right correspondences: {len(ptsL_R)}")

    def triangulate_points(P1, P2, pts1, pts2):
        if len(pts1) < 4 or len(pts2) < 4:
            return np.array([])
        pts4d = cv.triangulatePoints(P1, P2, pts1.T, pts2.T)
        pts3d = (pts4d[:3] / pts4d[3]).T
        valid = (pts3d[:, 2] > 0.1) & (pts3d[:, 2] < 10.0)
        return pts3d[valid]

    print("Triangulating...")
    points_FL = triangulate_points(P_F, P_L, ptsF_L, ptsL_F)
    points_FR = triangulate_points(P_F, P_R, ptsF_R, ptsR_F)
    points_LR = triangulate_points(P_L, P_R, ptsL_R, ptsR_L)

    all_points = np.vstack([points_FL, points_FR, points_LR])

    # Transform from OpenCV front camera frame to Blender world
    n = all_points.shape[0]
    points_hom = np.hstack([all_points, np.ones((n, 1))])
    M_front_inv = np.linalg.inv(extrinsic_F)
    points_world = (M_front_inv @ points_hom.T).T[:, :3]

    # Create and save point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_world)
    red_color = np.tile([1.0, 0.0, 0.0], (n, 1))
    pcd.colors = o3d.utility.Vector3dVector(red_color)

    o3d.io.write_point_cloud("reconstruction.ply", pcd)
    print(f"Reconstructed {n} points and saved to reconstruction.ply")

    # Save Blender-space camera centers
    def camera_center_from_extrinsic(E):
        R = E[:3, :3]
        t = E[:3, 3]
        return (-R.T @ t).tolist()

    camera_positions = {
        "Camera_Front": camera_center_from_extrinsic(extrinsic_F),
        "Camera_Left": camera_center_from_extrinsic(extrinsic_L),
        "Camera_Right": camera_center_from_extrinsic(extrinsic_R)
    }

    with open("camera_positions.json", "w") as f:
        json.dump(camera_positions, f)
    print("Saved camera positions to camera_positions.json")
    
    # Evaluate reconstruction
    print("\nEvaluating reconstruction quality...")
    metrics = evaluate_reconstruction(points_world, params)
        # Create visualization point cloud
    if metrics:
        # Create reconstruction point cloud (red)
        recon_pcd = o3d.geometry.PointCloud()
        recon_pcd.points = o3d.utility.Vector3dVector(points_world)
        recon_colors = np.tile([1.0, 0.0, 0.0], (len(points_world), 1))  # Red
        recon_pcd.colors = o3d.utility.Vector3dVector(recon_colors)

        # Camera positions (blue)
        cam_points = []
        for cam_name in ["Camera_Front", "Camera_Left", "Camera_Right"]:
            cam_points.append(camera_center_from_extrinsic(np.array(params[cam_name]["extrinsic"])))
        cam_points = np.array(cam_points)

        cam_pcd = o3d.geometry.PointCloud()
        cam_pcd.points = o3d.utility.Vector3dVector(cam_points)
        cam_colors = np.tile([0.0, 0.0, 1.0], (len(cam_points), 1))  # Blue
        cam_pcd.colors = o3d.utility.Vector3dVector(cam_colors)

        # Save evaluation (reconstruction + cameras)
        combined_pcd = recon_pcd + cam_pcd
        o3d.io.write_point_cloud("evaluation.ply", combined_pcd)
        print("Saved evaluation point cloud to evaluation.ply")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Matching pairs...
Left-Front correspondences: 6111
Front-Right correspondences: 5862
Left-Right correspondences: 4168
Triangulating...
Reconstructed 16141 points and saved to reconstruction.ply
Saved camera positions to camera_positions.json

Evaluating reconstruction quality...
Camera_Left: Generated 848931 ground truth points
Camera_Right: Generated 720304 ground truth points
Camera_Front: Generated 773210 ground truth points
Total ground truth points: 2342445
Downsampling ground truth points for efficiency

------------------------------------------------------------
Evaluation Metrics (units: centimeters)
------------------------------------------------------------
Matched Points      : 16141/16141
Mean Error          : 0.042 cm
Median Error        : 0.036 cm
RMSE                : 0.061 cm
Std Dev             : 0.04